<a href="https://colab.research.google.com/github/AmirJffr/DataScience-and-Machine-Learning-Master-SI/blob/main/Assignements/Part%206/Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/michalis0/DataScience_and_MachineLearning/blob/master/Assignements/Part%206/Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### DSML investigation

You are part of the Suisse Impossible Mission Force, or SIMF for short. You need to uncover a rogue agent that is trying to steal sensitive information.

Your mission, should you choose to accept it, is to find that agent before stealing any classified information. Good luck!

# Assignment part Six

### Mission Briefing

Welcome back to the **SIMF (Swiss Impossible Mission Force)**. Your objective remains the same: find the rogue agent before sensitive information is compromised.

### New Intelligence
We have received new intelligence that the rogue agent has attended various pop concerts. SIMF has tried checking the history of our remaining suspects on spotify, however none of them have any pop songs.

We need to dig deeper. To do so, we will build a basic recommender system using the spotify data to identify which of the remaining suspects is most likely to enjoy pop music.

#### Suspects remaining:
- **628854**
- **410319**
- **785994**






### Available Data
The only data you have is the spotify listening history of the suspects, and a list of user interactions with various songs.
The data may already contain some of the suspect interactions.

### Build a Recommender System
Using the data you have been provided, build a basic recommender system to identify which of the remaining suspects is most likely to enjoy pop music.

### Hints:
- Filter out the interactions data to not include the "skip" interactions and the interactions with a rating less than 3.
- Use the filtered interactions data to build a user-item matrix.
- Use cosine similarity to find songs similar to pop songs.
- Remove features that are not necessary for the recommender system.

## 1. Getting to know our data
- Interaction_ID: Unique identifier for each interaction
- User_ID: Unique identifier for each user
- File_Type: Type of file (e.g., audio, video)
- Interaction_Type: (Listen, Skip, Share, Like)
- Rating: User rating for the interaction (1-5 scale)
- Timestamp: Date and time of the interaction
- Age: Age of the user
- Gender: Gender of the user
- Subscription_Type: Type of subscription (Free, Premium)
- Title: Title of the song
- Artist: Artist of the song
- Album: Album of the song
- Genre: Genre of the song
- Release_Year: Year the song was released
- Lyrics: Lyrics of the song
- File_id: Unique identifier for each file


In [2]:
import pandas as pd
import os
# download the dataset from github
if os.path.exists("train_data.csv") == False:
    !wget https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/refs/heads/master/Assignements/Part%206/train_data.csv
data = pd.read_csv("train_data.csv")
data.tail(10)



--2025-11-13 23:57:24--  https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/refs/heads/master/Assignements/Part%206/train_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 892269 (871K) [text/plain]
Saving to: ‘train_data.csv’

train_data.csv      100%[===================>] 871.36K  --.-KB/s    in 0.03s   

2025-11-13 23:57:24 (26.3 MB/s) - ‘train_data.csv’ saved [892269/892269]



,Interaction_ID,User_ID,File_Type,Interaction_Type,Rating,Timestamp,Age,Gender,Subscription_Type,Title,Artist,Album,Genre,Release_Year,Lyrics,File_ID,Sentiment,Target
4988,I04991,U0053,Audio,Listen,2,2025-08-31 01:58:30.422989,48,F,Premium,Audio_Song_1781,Dr. Dre,Album_61,Electronic,2012,I get by with a little help from my friends,A1781,0.416,0
4989,I04992,U0393,Audio,Share,4,2023-04-17 23:49:30.422989,54,Other,Free,Audio_Song_1809,Lorde,Album_94,Pop,1994,"Nothing else matters, never cared for what the...",A1809,0.548,1
4990,I04993,U0335,Video,Like,5,2023-02-19 11:55:30.422989,53,F,Premium,Video_Song_941,Tina Turner,Album_5,Classical,2003,"Rolling in the deep, you had my heart inside o...",V941,0.103,1
4991,I04994,U0342,Audio,Like,1,2022-12-29 13:33:30.422989,58,Other,Free,Audio_Song_2088,Tina Turner,Album_56,Classical,1999,"Cause baby you're a firework, come on show 'em...",A2088,-0.235,0
4992,I04995,U0161,Video,Skip,3,2023-03-23 16:15:30.422989,41,F,Premium,Video_Song_1282,U2,Album_92,Jazz,2012,"All the single ladies, all the single ladies",V1282,0.340,0
4993,I04996,U0289,Audio,Like,4,2024-01-31 15:23:30.422989,45,M,Premium,Audio_Song_587,Katy Perry,Album_87,Rock,2022,"I want to hold your hand, I want to hold your ...",A587,-0.339,1
4994,I04997,U0311,Video,Listen,1,2024-10-22 21:13:30.422989,15,M,Premium,Video_Song_2346,Queen,Album_46,Pop,1991,"Hello from the other side, I must have called ...",V2346,0.634,0
4995,I04998,U0411,Audio,Skip,3,2024-01-17 03:22:30.422989,53,M,Free,Audio_Song_2357,Travis Scott,Album_19,Rock,1995,"Don't stop believin', hold on to that feeling",A2357,-0.015,0
4996,I04999,U0110,Audio,Skip,1,2024-03-13 01:10:30.422989,19,M,Premium,Audio_Song_1745,Miley Cyrus,Album_83,Hip-Hop,2025,"Cause baby you're a firework, come on show 'em...",A1745,-0.464,0
4997,I05000,U0288,Audio,Listen,4,2023-06-10 14:30:30.422989,48,F,Premium,Audio_Song_1089,Lady Gaga,Album_10,Classical,2018,"Nothing else matters, never cared for what the...",A1089,-0.109,1


### Data cleaning keep only the interactions with a rating 3 or above and that do not consist in a 'skip'

In [3]:
data = data[(data['Rating'] >= 3) & (data['Interaction_Type'] != 'Skip')]
data.head(1)

,Interaction_ID,User_ID,File_Type,Interaction_Type,Rating,Timestamp,Age,Gender,Subscription_Type,Title,Artist,Album,Genre,Release_Year,Lyrics,File_ID,Sentiment,Target
0,I00001,U0494,Audio,Listen,4,2023-06-05 00:05:30.389872,51,M,Premium,Audio_Song_1825,Bruno Mars,Album_90,Pop,2020,I get by with a little help from my friends,A1825,-0.735,1


## 2. Build a user-item matrix
Now that we have cleaned the data, we can build a user-item matrix. This matrix will help us understand the interactions between users and songs.

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# build a user-item interaction matrix, you can use pivot_table from pandas
interaction_matrix = data.pivot_table(index='User_ID', columns='Title', values='Rating', aggfunc='mean')
interaction_matrix.fillna(0, inplace=True)


In [5]:
interaction_matrix.head()

Title,Audio_Song_10,Audio_Song_1000,Audio_Song_1001,Audio_Song_1003,Audio_Song_1004,Audio_Song_1006,Audio_Song_1007,Audio_Song_1009,Audio_Song_101,Audio_Song_1014,...,Video_Song_977,Video_Song_98,Video_Song_980,Video_Song_983,Video_Song_987,Video_Song_99,Video_Song_990,Video_Song_992,Video_Song_994,Video_Song_999
User_ID,,,,,,,,,,,,,,,,,,,,,
410319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
628854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
785994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 4. identifying the recommendations for our 3 suspects
Now that we have built the recommender system, we can use it to identify which of the remaining suspects is most likely to enjoy pop music.
- For each suspect, use the songs they have interacted with.
- Find similar songs using the recommender system.
- We will use cosine similarity to find similar songs, and set k = 5.

- **628854** has listened to songs: Video_Song_1683, Audio_Song_2372, Audio_Song_280, Audio_Song_1825, Video_Song_1825, Video_Song_1006
- **410319** has listened to songs: Audio_Song_717, Audio_Song_2372, Audio_Song_1481, Video_Song_572
- **785994** has listened to songs: Audio_Song_1733, Audio_Song_389, Video_Song_2500, Video_Song_660



In [6]:
from sklearn.metrics.pairwise import cosine_similarity
item_similarity = cosine_similarity(interaction_matrix.T)

def get_recommendations(user_id, data_matrix, item_similarity, top_n=5):
    sim_df = pd.DataFrame(item_similarity,
                          index=data_matrix.columns,
                          columns=data_matrix.columns)

    if user_id not in data_matrix.index:
        return []

    user_ratings = data_matrix.loc[user_id]
    listened = user_ratings[user_ratings > 0].index.tolist()

    recs = []

    for song in listened:
        if song in sim_df.index:
            recs += sim_df.loc[song].sort_values(ascending=False).iloc[1:6].index.tolist()

    recs = [r for r in dict.fromkeys(recs) if r not in listened]

    return recs[:top_n]



### Find the recommendations for each suspect and identify which suspect is most likely to enjoy pop music.

In [8]:
suspects_songs = {
    628854: ["Video_Song_1683", "Audio_Song_2372", "Audio_Song_280",
             "Audio_Song_1825", "Video_Song_1825", "Video_Song_1006"],
    410319: ["Audio_Song_717", "Audio_Song_2372", "Audio_Song_1481", "Video_Song_572"],
    785994: ["Audio_Song_1733", "Audio_Song_389", "Video_Song_2500", "Video_Song_660"]
}

song_genres = data[['Title','Genre']].drop_duplicates().set_index('Title')

pop_scores = {}
for suspect, songs in suspects_songs.items():

    temp_matrix = interaction_matrix.copy()
    temp_matrix.loc[suspect] = 0

    for s in songs:
        if s in temp_matrix.columns:
            temp_matrix.loc[suspect, s] = 5

    recs = get_recommendations(suspect, temp_matrix, item_similarity, top_n=5 )
    pop_list = [r for r in recs
                if r in song_genres.index and song_genres.loc[r,"Genre"]=="Pop"]

    pop_scores[suspect] = len(pop_list)

print("Pop scores :", pop_scores)
print("Suspect le plus susceptible d'aimer la Pop :", max(pop_scores, key=pop_scores.get))


Pop scores : {628854: 0, 410319: 2, 785994: 3}
Suspect le plus susceptible d'aimer la Pop : 785994


## Your investigation is complete. The SIMF task force extends their deepest gratitude for your unwavering dedication and service.

**Remember to complete the quiz and submit your code (this notebook) on Moodle before the deadline. Your mission isn’t over until all tasks are finished!**